In [1]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow_io as tfio
from collections import Counter

2023-04-27 22:38:02.998297: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-27 22:38:03.609751: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
POS_UP = os.path.join('YANDEX_DATASET', 'speech_commands_train', 'вверх')
POS_UP


'YANDEX_DATASET/speech_commands_train/вверх'

In [5]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [3]:
files_positive_up = tf.data.Dataset.list_files(POS_UP+'/*.wav')
files_positive_up

2023-04-27 22:39:00.870186: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-27 22:39:00.873257: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<_ShuffleDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [4]:
positives = tf.data.Dataset.zip((files_positive_up, tf.data.Dataset.from_tensor_slices(tf.ones(len(files_positive_up)))))


<_ZipDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float32, name=None))>

In [6]:
for file in os.listdir(os.path.join('YANDEX_DATASET', 'speech_commands_train', 'вверх')):
    tensor_wave = load_wav_16k_mono(os.path.join('YANDEX_DATASET', 'speech_commands_train', 'вверх', file))


2023-04-27 22:41:01.036456: I tensorflow_io/core/kernels/cpu_check.cc:128] Your CPU supports instructions that this TensorFlow IO binary was not compiled to use: AVX2 FMA


In [7]:
tensor_wave

<tf.Tensor: shape=(64000,), dtype=float32, numpy=
array([0.0000000e+00, 1.7656580e-10, 2.6549318e-12, ..., 5.0600875e-06,
       5.5396973e-05, 3.6665053e-05], dtype=float32)>

In [8]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

In [13]:
filepath, label = positives.as_numpy_iterator().next()

2023-04-27 22:51:01.669627: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype float and shape [2497]
	 [[{{node Placeholder/_7}}]]
2023-04-27 22:51:01.669799: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype float and shape [2497]
	 [[{{node Placeholder/_7}}]]


In [18]:
spectrogram_list = []
for i in range(10):
    filepath, label = positives.as_numpy_iterator().next()
    spectrogram, label = preprocess(filepath, label)
    spectrogram_list.append(spectrogram)    

In [21]:
spectrogram_list

[<tf.Tensor: shape=(1491, 257, 1), dtype=float32, numpy=
 array([[[1.62006938e-04],
         [1.09506524e-04],
         [2.98935920e-04],
         ...,
         [9.33556021e-06],
         [3.93098026e-06],
         [4.75754496e-06]],
 
        [[7.70660117e-05],
         [7.89172409e-05],
         [2.29998011e-04],
         ...,
         [1.32257055e-05],
         [8.26496762e-06],
         [7.39446841e-06]],
 
        [[5.29143144e-05],
         [1.31103225e-04],
         [2.73093145e-04],
         ...,
         [1.28712663e-05],
         [1.14099757e-05],
         [1.10198162e-05]],
 
        ...,
 
        [[1.61990145e-04],
         [1.25806415e-04],
         [1.21331999e-04],
         ...,
         [4.42351666e-06],
         [2.52311338e-06],
         [2.34206527e-06]],
 
        [[1.68151397e-04],
         [1.51767890e-04],
         [1.27779436e-04],
         ...,
         [2.87226203e-06],
         [2.31277181e-06],
         [3.19268293e-06]],
 
        [[2.26399643e-04],
      

In [25]:
spectrogram_list[5]

<tf.Tensor: shape=(1491, 257, 1), dtype=float32, numpy=
array([[[1.44666585e-04],
        [2.63753871e-04],
        [3.18601116e-04],
        ...,
        [3.95447387e-06],
        [7.90128161e-06],
        [1.04492647e-05]],

       [[2.40924550e-04],
        [2.09564299e-04],
        [1.94995635e-04],
        ...,
        [3.72358454e-06],
        [7.18363754e-06],
        [8.75779369e-06]],

       [[1.06285726e-04],
        [1.87117257e-04],
        [1.48655919e-04],
        ...,
        [3.22607320e-06],
        [6.14402734e-06],
        [7.00545934e-06]],

       ...,

       [[3.54610238e-05],
        [6.96405623e-05],
        [1.76035799e-04],
        ...,
        [6.87489546e-06],
        [6.99289603e-06],
        [7.19430682e-06]],

       [[4.38989373e-05],
        [1.18198448e-04],
        [1.64218392e-04],
        ...,
        [7.58286706e-06],
        [8.05658510e-06],
        [8.19281559e-06]],

       [[1.86947829e-04],
        [1.89857397e-04],
        [1.04530154e-04]

In [14]:
spectrogram, label = preprocess(filepath, label)

In [15]:
spectrogram

<tf.Tensor: shape=(1491, 257, 1), dtype=float32, numpy=
array([[[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       [[0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00],
        ...,
        [0.0000000e+00],
        [0.0000000e+00],
        [0.0000000e+00]],

       ...,

       [[5.4716205e-05],
        [1.1827024e-04],
        [1.3531593e-04],
        ...,
        [4.7265962e-06],
        [4.4732210e-06],
        [5.0596427e-06]],

       [[7.6897486e-05],
        [1.0262005e-04],
        [1.2590681e-04],
        ...,
        [2.5223180e-06],
        [4.0563491e-06],
        [3.4145160e-06]],

       [[1.5908261e-04],
        [1.9430245e-04],
        [1.1687243e-04],
        ...,
        [2.1281996